In [38]:
from mbientlab.metawear import MetaWear, libmetawear
import mbientlab.metawear.cbindings as cbindings
from ctypes import Structure, c_void_p, cast, POINTER, c_int, c_float
from threading import Event
from time import sleep

In [36]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'

# Create and connect the device
device = MetaWear(device_address)
device.connect()

In [39]:
class Data_array(Structure):
    _fields_ = [("count", c_int),
                ("values", POINTER(c_float))]

# Callback for data handling
def data_handler(context, data):
    value = cast(data.contents.value, POINTER(Data_array)).contents
    print(value)


In [40]:
e = Event()
callback = cbindings.FnVoid_VoidP_DataP(data_handler)

# Callback for processor creation
def processor_created(context, pointer):
    global processor
    processor = pointer
    e.set()

# Function pointer
fn_wrapper = cbindings.FnVoid_VoidP_VoidP(processor_created)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)

# Fuse the signals together
signals = (c_void_p * 1)()
signals[0] = gyro
libmetawear.mbl_mw_dataprocessor_fuser_create(acc, signals, 1, None, fn_wrapper)
e.wait()

# Subscribe to it
libmetawear.mbl_mw_datasignal_subscribe(processor, None, callback)

# Start the gyro and acc
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)
libmetawear.mbl_mw_acc_start(device.board)

# Stream data for a duration (e.g., 30 seconds)
sleep(5)

# Stop and clean up
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(processor)
libmetawear.mbl_mw_debug_disconnect(device.board)
print("Disconnected and cleaned up")

OSError: exception: access violation reading 0x0000000000000000

In [34]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

we are disconnected!


In [14]:
print(cbindings.DataTypeId.CARTESIAN_FLOAT)

2
